In [13]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 140E-0919

 Directory of c:\Users\lit\Desktop\heartsound2022

12/17/2024  12:42 AM    <DIR>          .
12/17/2024  12:22 AM    <DIR>          ..
12/17/2024  01:02 AM            56,183 detect.ipynb
12/17/2024  12:18 AM           266,560 heart-sound-v2-85c311.ipynb
12/17/2024  12:18 AM           284,902 heart-sound-v2-new.ipynb
12/17/2024  12:48 AM           161,216 heart-sound-v3.ipynb
12/17/2024  12:42 AM             9,795 id_patient.csv
12/17/2024  12:18 AM               870 README.md
12/17/2024  12:18 AM                75 requirements.txt
12/17/2024  12:18 AM            36,887 slpit_wav_each_folder.ipynb
12/17/2024  12:18 AM             4,997 slpit_wav_each_folder.py
12/17/2024  12:41 AM    <DIR>          the-circor-digiscope-phonocardiogram-dataset-1.0.3
12/17/2024  12:37 AM       471,283,852 the-circor-digiscope-phonocardiogram-dataset-1.0.3.zip
12/17/2024  12:18 AM           117,905 training_data.csv
12/17/2024  12:42 AM   

**Khai báo thư viện cần sử dụng**

In [14]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer
import pickle
import cv2

**Tạo các function tạo giá trị nhiễu , kéo giãn âm thanh. Để tăng data**

In [15]:
def noise(data,index = 0.05):
    noise_amp = index*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

# def pitch(data, sampling_rate, pitch_factor=0.8):
#     return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

def higher_speed(data):
    return librosa.effects.time_stretch(data, rate=1.2)

def lower_speed(data):
    return librosa.effects.time_stretch(data, rate=0.8)

**Function hiển thị biểu đồ hàm loss và accuracy sau quá trình train**

**Fucntion để lấy data từ file âm thanh và nhãn**

In [16]:

def get_data(name_files,audio,time_limit,sr):

    segment_length = time_limit * sr
    num_segments = (len(audio) // segment_length)
    x = []
    y = []
    for i in range(num_segments):
        start_sample = i * segment_length
        end_sample = (i + 1) * segment_length
        
        segment = audio[start_sample:end_sample]

        create_data(segment,n_fft,n_mels,hop_length)
        
        label = name_files.split('/')[2]
        
        x.append(create_data(segment,n_fft,n_mels,hop_length))

        y.append(label)
    # print(len(x))
    return x,y
def create_data (audio,n_fft,n_mels,hop_length):
    mel_spectrogram = librosa.feature.melspectrogram(y=audio,n_fft= n_fft ,n_mels=n_mels,hop_length=hop_length)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram ,ref=np.max)
    mel_spectrogram_db = mel_spectrogram_db + np.abs(np.min(mel_spectrogram_db))
    mel_spectrogram_db= cv2.resize(mel_spectrogram_db,output_size[:2])
    mel_spectrogram_db= np.reshape(mel_spectrogram_db,output_size)
    return mel_spectrogram_db

 **Tham số để để cho quá trình biển đổi , trích xuất âm thanh thô sang mel-spectrogram**

In [17]:
duration = 2
time_limit = duration
sr = 4000
n_fft = 512
n_mels= 128
hop_length = 256
output_size = (75,75,1)

**Lấy data từ file wav có nhãn là Absent**

In [18]:

x = []
y  = []
x_absent_test = []
y_absent_test =[]
dem = 0
name_folders = "training_data_all/test/Absent"
for file in os.listdir(name_folders):
    name_files = os.path.join(name_folders,file)

    time_start = float((file.split('_')[1]))

    time_end = (float((file.split('_')[2]))-float((file.split('_')[1])))
    audio, sr = librosa.load(name_files, sr=sr,offset = time_start,duration = time_end)

    x,y = get_data(name_files,audio,time_limit = time_limit,sr = sr)

    x_absent_test.extend(x)
    y_absent_test.extend(y)
print(np.array(x_absent_test).shape)
print(np.array(y_absent_test).shape)

(2830, 75, 75, 1)
(2830,)


In [19]:
x = []
y  = []
x_present_test = []
y_present_test =[]
dem = 0
name_folders = "training_data_all/test/Present"
for i in range(2):
    for file in os.listdir(name_folders):
        name_files = os.path.join(name_folders,file)

        time_start = float((file.split('_')[1]))

        time_end = (float((file.split('_')[2]))-float((file.split('_')[1])))
        audio, sr = librosa.load(name_files, sr=sr,offset = time_start,duration = time_end)
        
        if(i==0):
            x,y=get_data(name_files,audio,time_limit = time_limit,sr = sr)
        elif(i==1):
            x,y=get_data(name_files,noise(audio),time_limit = time_limit,sr = sr)
        x_present_test.extend(x)
        y_present_test.extend(y)
print(np.array(x_present_test).shape)
print(np.array(y_present_test).shape)

(1388, 75, 75, 1)
(1388,)


**Lấy data từ file wav có nhãn là Present**

In [20]:
x = []
y  = []
x_unknown_test = []
y_unknown_test =[]
dem = 0
name_folders = "training_data_all/test/Unknown"
for i in range(5):
    for file in os.listdir(name_folders):
        name_files = os.path.join(name_folders,file)

        time_start = float((file.split('_')[1]))

        time_end = (float((file.split('_')[2]))-float((file.split('_')[1])))
        audio, sr = librosa.load(name_files, sr=sr)
        
        if(i==0):
            x,y=get_data(name_files,audio,time_limit = time_limit,sr = sr)
        elif(i==1):
            x,y=get_data(name_files,noise(audio),time_limit = time_limit,sr = sr)
        elif(i==2):
            x,y=get_data(name_files,lower_speed(audio),time_limit = time_limit,sr = sr)
        elif(i==3):
            x,y=get_data(name_files,higher_speed(audio),time_limit = time_limit,sr = sr)
        elif(i==4):
            index = 0.1
            x,y=get_data(name_files,noise(audio),time_limit = time_limit,sr = sr)
        x_unknown_test.extend(x)
        y_unknown_test.extend(y)
print(np.array(x_unknown_test).shape)
print(np.array(y_unknown_test).shape)

(1921, 75, 75, 1)
(1921,)


In [21]:

x = []
y  = []
x_absent_train = []
y_absent_train =[]
dem = 0
name_folders = "training_data_all/train/Absent"
for file in os.listdir(name_folders):
    name_files = os.path.join(name_folders,file)

    time_start = float((file.split('_')[1]))

    time_end = (float((file.split('_')[2]))-float((file.split('_')[1])))
    audio, sr = librosa.load(name_files, sr=sr,offset = time_start,duration = time_end)

    x,y = get_data(name_files,audio,time_limit = time_limit,sr = sr)

    x_absent_train.extend(x)
    y_absent_train.extend(y)
print(np.array(x_absent_train).shape)
print(np.array(y_absent_train).shape)

(5094, 75, 75, 1)
(5094,)


In [22]:
x = []
y  = []
x_present_train = []
y_present_train =[]
dem = 0
name_folders = "training_data_all/train/Present"
for i in range(2):
    for file in os.listdir(name_folders):
        name_files = os.path.join(name_folders,file)

        time_start = float((file.split('_')[1]))

        time_end = (float((file.split('_')[2]))-float((file.split('_')[1])))
        audio, sr = librosa.load(name_files, sr=sr,offset = time_start,duration = time_end)
        
        if(i==0):
            x,y=get_data(name_files,audio,time_limit = time_limit,sr = sr)
        elif(i==1):
            x,y=get_data(name_files,noise(audio),time_limit = time_limit,sr = sr)
        x_present_train.extend(x)
        y_present_train.extend(y)
print(np.array(x_present_train).shape)
print(np.array(y_present_train).shape)

(2878, 75, 75, 1)
(2878,)


In [23]:
x = []
y  = []
x_unknown_train = []
y_unknown_train =[]
dem = 0
name_folders = "training_data_all/train/Unknown"
for i in range(5):
    for file in os.listdir(name_folders):
        name_files = os.path.join(name_folders,file)

        time_start = float((file.split('_')[1]))

        time_end = (float((file.split('_')[2]))-float((file.split('_')[1])))
        audio, sr = librosa.load(name_files, sr=sr)
        
        if(i==0):
            x,y=get_data(name_files,audio,time_limit = time_limit,sr = sr)
        elif(i==1):
            x,y=get_data(name_files,noise(audio),time_limit = time_limit,sr = sr)
        elif(i==2):
            x,y=get_data(name_files,lower_speed(audio),time_limit = time_limit,sr = sr)
        elif(i==3):
            x,y=get_data(name_files,higher_speed(audio),time_limit = time_limit,sr = sr)
        elif(i==4):
            index = 0.1
            x,y=get_data(name_files,noise(audio),time_limit = time_limit,sr = sr)
        x_unknown_train.extend(x)
        y_unknown_train.extend(y)
print(np.array(x_unknown_train).shape)
print(np.array(y_unknown_train).shape)

(3020, 75, 75, 1)
(3020,)


In [24]:
x_array = x_absent_train
y_array = y_absent_train
x_absent_train_split = np.array_split(x_array, 5)
y_absent_train_split = np.array_split(y_array, 5)
print(x_absent_train_split[4].shape)
print(y_absent_train_split[4].shape)

x_array = x_present_train
y_array = y_present_train
x_present_train_split = np.array_split(x_array, 5)
y_present_train_split = np.array_split(y_array, 5)
print(x_present_train_split[4].shape)
print(y_present_train_split[4].shape)

x_array = x_unknown_train
y_array = y_unknown_train
x_unknown_train_split = np.array_split(x_array, 5)
y_unknown_train_split = np.array_split(y_array, 5)
print(x_unknown_train_split[4].shape)
print(y_unknown_train_split[4].shape)

(1018, 75, 75, 1)
(1018,)
(575, 75, 75, 1)
(575,)
(604, 75, 75, 1)
(604,)


In [25]:
x_train =               (x_absent_train_split,
                        x_present_train_split,
                        x_unknown_train_split)
y_train =(  y_absent_train_split,
            y_present_train_split,
            y_unknown_train_split)
x_test = np.concatenate((x_absent_test,
                         x_present_test,
                         x_unknown_test),axis=0)
y_test = np.concatenate((y_absent_test,
                         y_present_test,
                         y_unknown_test),axis= 0)

file = open('data_spectrogram.csv','wb')
pickle.dump((x_train,x_test,y_train,y_test),file)
file.close()